In [ ]:
!pip install git+https://github.com/CatalinVoss/GuidedLDA
import guidedlda

  Cloning https://github.com/CatalinVoss/GuidedLDA to /tmp/pip-req-build-us7b9fen
  Running command git clone --filter=blob:none --quiet https://github.com/CatalinVoss/GuidedLDA /tmp/pip-req-build-us7b9fen
  Resolved https://github.com/CatalinVoss/GuidedLDA to commit 9440ebd5ae09ba4187880eadb842d23d743ecb4d
  Preparing metadata (setup.py) ... done


In [20]:
import warnings # import the warnings
warnings.filterwarnings("ignore") #ignoring the warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


#!python -m pip install --user --upgrade pip
#!pip install tmtoolkit  #installing the tmtoolkit
#!pip install pyLDAvis
#!pip install statsmodels --upgrade
#!pip install pandas --upgrade

#!pip install sentence-transformers 


import numpy as np # importing the numpy
import pandas as pd #importing the pandas
import re, nltk, gensim # importing re, nltk, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation # importing lda using sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer #importing Count and Tfidf
from sentence_transformers import SentenceTransformer #importing senetence transfomer



# Plotting tools
import pyLDAvis #importing pyldavis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt #importing the matplot
%matplotlib inline

#performing preprocessing on data
import string  #importing the string 
from nltk.corpus import wordnet #wordnet
from nltk.tokenize import word_tokenize #importing the word_tokenize
from nltk.stem import WordNetLemmatizer #importing the WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
#reading the dataset 
df= pd.read_csv('all-data (1).csv',header=None,
                 names=['sentiment','News'],encoding='latin-1')

#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text)
    text=re.compile('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+').sub('', text) 
    text=re.compile('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    
    return text


  
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)



#LEMMATIZATION
# Initialize the lemmatizer


from nltk.corpus import stopwords 
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)


def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))



In [22]:
df['Clean_text'] = df['News'].apply(lambda x: finalpreprocess(x))
df['Clean_text']

0       accord gran company plan move production russi...
1       technopolis plan develop stage area less squar...
2       international electronic industry company elco...
3       new production plant company would increase ca...
4       accord company update strategy year basware ta...
                              ...                        
4841    london marketwatch share price end lower londo...
4842    rinkuskiai beer sale fell per cent million lit...
4843    operating profit fell eur mn eur mn include ve...
4844    net sale paper segment decrease eur mn second ...
4845    sale finland decreased january sale outside fi...
Name: Clean_text, Length: 4846, dtype: object

In [23]:
df=df.drop_duplicates(subset = ['Clean_text'],keep = 'last').reset_index(drop = True)
from collections import OrderedDict

df['Desired'] = (df['Clean_text'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))

In [24]:
df['Desired']=df['Desired'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

# Gudied LDA CountVectorizer

In [55]:
def C_V(X): #define function for for Countvectorizer
    vectorizer = CountVectorizer(analyzer='word',       
                                 min_df=10,                        # minimum reqd occurences of a word 
                                 stop_words='english',             # remove stop words
                                 lowercase=True,                   # convert all words to lowercase
                                 token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                                )

    data_vectorized = vectorizer.fit_transform(X)  #fit and transforming the text and converting into vectors
      # get all unique words in the corpus
    vocab = vectorizer.get_feature_names()

    return vectorizer,data_vectorized,vocab

In [61]:
import tmtoolkit #importing the tmtoolkit
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim
def topic_model_coherence_generator(topic_num_start=2,    #def to calculate the coherence score
                                    topic_num_end=26,
                                    norm_corpus='',
                                    cv_matrix='',
                                    cv=''):
    norm_corpus_tokens = [doc.split() for doc in norm_corpus]
    models = []
    coherence_scores = []

    for i in range(topic_num_start, topic_num_end):
        #print(i)
        cur_lda = LatentDirichletAllocation(n_components=i,
                                            random_state=0)
        cur_lda.fit_transform(cv_matrix)
        cur_coherence_score = metric_coherence_gensim(
            measure='c_v',
            top_n=5,
            topic_word_distrib=cur_lda.components_,
            dtm=cv.fit_transform(norm_corpus),
            vocab=np.array(cv.get_feature_names()),
            texts=norm_corpus_tokens)
        models.append(cur_lda)
        coherence_scores.append(np.mean(cur_coherence_score))
    return models, coherence_scores

In [57]:
def show_topics(vectorizer='', lda_model='', n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [58]:
def Glda_topic_modelling_cv(X):
  
  vectorizer,data_vectorized,vocab=C_V(X)

  models, coherence_scores = topic_model_coherence_generator(
                                                            norm_corpus=X, #text
                                                            cv=vectorizer, #CountVectorizer
                                                            cv_matrix=data_vectorized)#data from CountVectorizer
  ts=2
  te=26
  coherence_df = pd.DataFrame({ #creating a dataframe 
                              'TOPIC_NUMBER': [str(i) for i in range(ts, te)],
                              'COHERENCE_SCORE': np.round(coherence_scores, 4)})
  n=coherence_df.sort_values(by=["COHERENCE_SCORE"], ascending=False).head(1) 
  n_topics=int(list(n['TOPIC_NUMBER'].values)[0])

  # Instantiate LDA Model with parameters
  lda_model = LatentDirichletAllocation(n_components=n_topics,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                      )
  doc_topic_matrix = lda_model.fit_transform(data_vectorized)




  topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=100)        

  # Topic - Keywords Dataframe
  df_topic_keywords = pd.DataFrame(topic_keywords)
  df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
  df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

  topic={} #creating the empty dict for topics 
  for i in range(df_topic_keywords.shape[0]): 
      topic[str(i)] =df_topic_keywords.iloc[i].tolist() #giving the keys has topic number and value has key words


  seed_topic_list=[] #creating empty list for topics 
  for i in topic.values(): 
      seed_topic_list.append(i)#appending the topics 

  import guidedlda #importing the Guided lda
  tf_feature_names = vocab

  word2id = dict((v, idx) for idx, v in enumerate(tf_feature_names))


  seed_list=[]
  for i in range(len(seed_topic_list)): #appending the vocad to key topic words
      seed_list.append([x for x in seed_topic_list[i] if x in list(word2id.keys())])


  model = guidedlda.GuidedLDA(n_topics=n_topics, n_iter=100, random_state=100, refresh=20,alpha=0.01,eta=0.01)

  seed_topics = {}
  for t_id, st in enumerate(seed_list):
      for word in st:
          seed_topics[word2id[word]] = t_id


  model=model.fit(data_vectorized, seed_topics=seed_topics, seed_confidence=0.15) #fitting the gudied lda


  pyLDAvis.enable_notebook()
  panel = pyLDAvis.sklearn.prepare(model, data_vectorized, vectorizer, mds='tsne')
  return panel




In [ ]:
!pip install pandas --upgrade

In [60]:
%%time
panel=Glda_topic_modelling_cv(df['Desired'])

2


INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 2000 documents


3


INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 3000 documents


4


...

24


INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 4000 documents


25


INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 4000 documents
INFO:guidedlda:n_documents: 4755
INFO:guidedlda:vocab_size: 946
INFO:guidedlda:n_words: 37173
INFO:guidedlda:n_topics: 10
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -435003
INFO:guidedlda:<20> log likelihood: -256842
INFO:guidedlda:<40> log likelihood: -251586
INFO:guidedlda:<60> log likelihood: -249009
INFO:guidedlda:<80> log likelihood: -247193
INFO:guidedlda:<99> log likelihood: -245682


CPU times: user 4min 28s, sys: 1.45 s, total: 4min 30s
Wall time: 4min 46s


In [39]:
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
8      32.420952   8.115230       1        1  15.689795
9       0.648955  71.707100       2        1  13.665547
7      71.019684   9.971006       3        1   9.902266
0       3.785981  33.185135       4        1   9.362171
5     -42.666500  -9.222070       5        1   9.099056
2      -3.621689 -43.526516       6        1   9.075243
4      -3.632402  -4.171708       7        1   8.674842
3     -36.584400  35.804413       8        1   8.666368
1      44.391380  50.615383       9        1   8.331558
6      40.235809 -31.591665      10        1   7.533155, topic_info=            Term        Freq       Total Category  logprob  loglift
277          eur  832.000000  832.000000  Default  30.0000  30.0000
663       profit  387.000000  387.000000  Default  29.0000  29.0000
541          net  377.000000  377.000000  Default  28.0000  28.0000
595          oyj  221.000000  221.000000  Default  27.0000  27.0000
316      finnish  509.000000  509.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
803        start   20.967198  104.071458  Topic10  -4.8945   0.9837
741          say   27.254214  590.573583  Topic10  -4.6323  -0.4900
254     employee   19.919362   84.660366  Topic10  -4.9458   1.1389
760      service   20.967198  275.691892  Topic10  -4.8945   0.0095
179  corporation   19.919362  109.136783  Topic10  -4.9458   0.8849

[615 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1         8  1.034450        abp
2         1  0.896235     access
2         3  0.112029     access
3         1  0.405419  accessory
3         9  0.608128  accessory
...     ...       ...        ...
944       8  0.596483  yesterday
945       2  0.089758        yit
945       3  0.059838        yit
945       6  0.718060        yit
945       7  0.119677        yit

[1433 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 8, 1, 6, 3, 5, 4, 2, 7])

# Gudied LDA TFVectorizer 

In [ ]:
def TF_V(X): #define function for for Countvectorizer
    vectorizer = TfidfVectorizer(analyzer='word',       
                                 min_df=10,                        # minimum reqd occurences of a word 
                                 stop_words='english',             # remove stop words
                                 lowercase=True,                   # convert all words to lowercase
                                 token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                                )

    data_vectorized = vectorizer.fit_transform(X)  #fit and transforming the text and converting into vectors
      # get all unique words in the corpus
    vocab = vectorizer.get_feature_names()

    return vectorizer,data_vectorized,vocab

In [ ]:
import tmtoolkit #importing the tmtoolkit
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim
def topic_model_coherence_generator(topic_num_start=2,    #def to calculate the coherence score
                                    topic_num_end=26,
                                    norm_corpus='',
                                    cv_matrix='',
                                    cv=''):
    norm_corpus_tokens = [doc.split() for doc in norm_corpus]
    models = []
    coherence_scores = []

    for i in range(topic_num_start, topic_num_end):
        #print(i)
        cur_lda = LatentDirichletAllocation(n_components=i,
                                            random_state=0)
        cur_lda.fit_transform(cv_matrix)
        cur_coherence_score = metric_coherence_gensim(
            measure='c_v',
            top_n=5,
            topic_word_distrib=cur_lda.components_,
            dtm=cv.fit_transform(norm_corpus),
            vocab=np.array(cv.get_feature_names()),
            texts=norm_corpus_tokens)
        models.append(cur_lda)
        coherence_scores.append(np.mean(cur_coherence_score))
    return models, coherence_scores

In [ ]:
def show_topics(vectorizer='', lda_model='', n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [ ]:
def Glda_topic_modelling_cv(X):
  
  vectorizer,data_vectorized,vocab=TF_V(X) #TFIDF VECTORS

  models, coherence_scores = topic_model_coherence_generator(
                                                            norm_corpus=X, #text
                                                            cv=vectorizer, #CountVectorizer
                                                            cv_matrix=data_vectorized)#data from CountVectorizer
  ts=2
  te=26
  coherence_df = pd.DataFrame({ #creating a dataframe 
                              'TOPIC_NUMBER': [str(i) for i in range(ts, te)],
                              'COHERENCE_SCORE': np.round(coherence_scores, 4)})
  n=coherence_df.sort_values(by=["COHERENCE_SCORE"], ascending=False).head(1) 
  n_topics=int(list(n['TOPIC_NUMBER'].values)[0])

  # Instantiate LDA Model with parameters
  lda_model = LatentDirichletAllocation(n_components=n_topics,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                      )
  doc_topic_matrix = lda_model.fit_transform(data_vectorized)




  topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=100)        

  # Topic - Keywords Dataframe
  df_topic_keywords = pd.DataFrame(topic_keywords)
  df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
  df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

  topic={} #creating the empty dict for topics 
  for i in range(df_topic_keywords.shape[0]): 
      topic[str(i)] =df_topic_keywords.iloc[i].tolist() #giving the keys has topic number and value has key words


  seed_topic_list=[] #creating empty list for topics 
  for i in topic.values(): 
      seed_topic_list.append(i)#appending the topics 

  import guidedlda #importing the Guided lda
  tf_feature_names = vocab

  word2id = dict((v, idx) for idx, v in enumerate(tf_feature_names))


  seed_list=[]
  for i in range(len(seed_topic_list)): #appending the vocad to key topic words
      seed_list.append([x for x in seed_topic_list[i] if x in list(word2id.keys())])


  model = guidedlda.GuidedLDA(n_topics=n_topics, n_iter=100, random_state=100, refresh=20,alpha=0.01,eta=0.01)

  seed_topics = {}
  for t_id, st in enumerate(seed_list):
      for word in st:
          seed_topics[word2id[word]] = t_id


  model=model.fit(data_vectorized.toarray().astype(int), seed_topics=seed_topics, seed_confidence=0.15) #fitting the gudied lda



  pyLDAvis.enable_notebook()
  panel = pyLDAvis.sklearn.prepare(model, data_vectorized.toarray().astype(int), vectorizer, mds='tsne')
  
  return score,panel




In [ ]:
%%time
panel=Glda_topic_modelling_cv(df['Clean_text'])